In [ ]:
import os
from typing import List
from langchain.text_splitter import RecursiveCharacterTextSplitter

from promptflow_vectordb.core.contracts import (
    EmbeddingModelType,
    StorageType,
    StoreCoreConfig,
)
from promptflow_vectordb.core.embeddingstore_core import EmbeddingStoreCore
import sys, os
sys.path.append('.')
from faiss_index import *

In [ ]:
for root, dirs, _ in os.walk('C:/Users/KPRAZUCH/OneDrive - HC-SC PHAC-ASPC/Documents/10k_sample'):
    for dir in dirs:
        if dir == <SUBSET>:
            print(f"{root}/{dir}")
            store = create_index(f"{root}/{dir}", 'faiss_index_store_10k')

In [ ]:
# if you need to clear (reset) the index:
# store.clear()
# and physically delete the faiss_index_store folder

In [ ]:
for result in store.search_by_text("Impact of diabetes on COVID-19 mortality and hospital outcomes", 3):
    print("Source:", result.metadata.get('source'))
    print("Score:", result.score)
    print("Text:", result.text)
    print("================================")

In [ ]:
from openai import OpenAI

def get_embedding_for_text(text):
    client = OpenAI()
    client.embeddings.create(
    model="text-embedding-ada-002",
    input=text,
    encoding_format="float"
    )

query_embedding = get_embedding_for_text("net zero")
for result in store.search_by_embedding(query_embedding, 3):
    print("Source:", result.metadata.get('source'))
    print("Score:", result.score)
    print("Text:", result.text)
    print("================================")